### this script construct conterfactual research profile which loosen the criteria in remove_foreignfund.ipynb
Assign publications to author country using full counting first, and then for each country, following publications are retained:
- articles that are not funded
- articles that are exclusively funded by the country itself
- **articles that are funded by the country itself as well as other countries**

The output of the script is the number of publications of countries in year and discipline

In [1]:
import pandas as pd

In [2]:
pubs_path="../../data/NationalFunding/Data/DerivedData/CleanedRawData/pub.pkl"

pub_filter_path="../../data/NationalFunding/Data/DerivedData/Derived/dependence/pub_noint_fund_exclusive_foreign.csv"

In [3]:
# Parameters
pubs_path = "../data/NationalFunding/Data/DerivedData/CleanedRawData/pub.pkl"
pub_filter_path = "../data/NationalFunding/Data/DerivedData/Derived/dependence/pub_no_exclusive_int_fund.csv"


In [4]:
pub_df=pd.read_pickle(pubs_path)
pub_df.head()

,id,year,dis,type,author,funder,author_distinct,funder_distinct,funded,IntCol,cofund,intfund
0,60694041,2017,9,1,"[Canada, United States]","[United States, United States]","[Canada, United States]",[United States],1,1,0,0
1,48573156,2012,89,1,[France],Not-Funded,[France],Not-Funded,0,0,0,0
2,55323671,2015,89,1,"[France, Chile]","[Chile, Chile, Chile, Chile, Chile]","[France, Chile]",[Chile],1,1,0,0
3,57633115,2016,31,1,"[Switzerland, United States]",[Switzerland],"[Switzerland, United States]",[Switzerland],1,1,0,0
4,64831413,2018,99,1,[China],"[China, China, China, China, China, China]",[China],[China],1,0,0,0


In [5]:
author_df=pub_df[['id','year','dis','author_distinct','funder_distinct']]
author_df=author_df.explode('author_distinct')

# detect whether the paper is exclusively funded by foreign grants

In [6]:
def assign_label(author,funders):
    label=1
    if funders=='Not-Funded':
        label=0
    elif author in funders:
        label=0
    return label
author_df['label']=author_df.apply(lambda x:assign_label(x.author_distinct,x.funder_distinct),axis=1)

In [7]:
author_df=author_df[author_df.label==0]
author_df=author_df.groupby(['author_distinct','year','dis']).size().reset_index()

In [8]:
author_df.columns=['cntry','year','dis','count']
author_df.to_csv(pub_filter_path,index=False)